# D2 - 02 - Advanced concepts

## Content
- Object-oriented programming (OOP)
- the `lambda` command (anonymous functions)
- iterators and generators

## Remember jupyter notebooks
- To run the currently highlighted cell, hold <kbd>&#x21E7; Shift</kbd> and press <kbd>&#x23ce; Enter</kbd>.
- To get help for a specific function, place the cursor within the function's brackets, hold <kbd>&#x21E7; Shift</kbd>, and press <kbd>&#x21E5; Tab</kbd>.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

## Object-oriented programming
A `class` is a blueprint for a container like data structure containing variables (**attributes**) and functions (**methods**). An `object` is an actual instance of a `class`.

**Example**: let's build a rotation matrix.

In [ ]:
class RotationMatrix(object):
    def __init__(self, angle):
        self.angle = angle % 360

rotation = RotationMatrix(90)
print(rotation)

In [ ]:
class RotationMatrix(object):
    def __init__(self, angle):
        self.angle = angle % 360
    def __repr__(self):
        return 'Rotation by %.2f°' % self.angle

rotation = RotationMatrix(90)
print(rotation)

In [ ]:
rotation2 = RotationMatrix(-90)
print(rotation2)

In [ ]:
rotation * rotation2

In [ ]:
class RotationMatrix(object):
    def __init__(self, angle):
        self.angle = angle % 360
    def __repr__(self):
        return 'Rotation by %.2f°' % self.angle
    def __mul__(self, other):
        return RotationMatrix(self.angle + other.angle)

rotation = RotationMatrix(90)
print(rotation)

rotation2 = RotationMatrix(-90)
print(rotation2)

print(rotation * rotation2)

In [ ]:
class RotationMatrix(object):
    def __init__(self, angle):
        self.angle = angle % 360
        self.rad = np.pi * self.angle / 180
        self.rotation_matrix = np.asarray([
            [np.cos(self.rad), -np.sin(self.rad)],
            [np.sin(self.rad), np.cos(self.rad)]])
    def __repr__(self):
        return 'Rotation by %.2f°' % self.angle
    def __mul__(self, other):
        return RotationMatrix(self.angle + other.angle)
    def rotate(self, vector):
        return np.dot(self.rotation_matrix, np.asarray(vector))

rotation = RotationMatrix(90)
print(rotation.rotate([1, 0]))

## The `lambda ` command
Python provides a mechanism to make anonymous functions. But frist, let's remember how to use a references to functions:

In [ ]:
def func(x):
    print('func here: ', x)

func([0, 1, 2])

In [ ]:
func_alias = func

func_alias([3, 4, 5])

We can create variables which are pointing to functions. With `lambda` we can create a function without a name and let some variable point to it:

In [ ]:
anonymous_func = lambda x: print('lambda here:', x)

anonymous_func([6, 7, 8])

The pattern is like this:

```Python
lambda variable: transform(variable)
```

Let's combine this with OOP and create a class for composing mathematical functions:

In [ ]:
class Function(object):
    def __init__(self, expression):
        self.expression = expression

f = Function(lambda x: x**2)

x = np.linspace(-2, 2, 100)
plt.plot(x, f.expression(x))

In [ ]:
class Function(object):
    def __init__(self, expression):
        self.expression = expression
    def __call__(self, x):
        return self.expression(x)

f = Function(lambda x: x**2)

plt.plot(x, f(x))

In [ ]:
class Function(object):
    def __init__(self, expression):
        self.expression = expression
    def __call__(self, x):
        return self.expression(x)
    def __or__(self, other):
        return Function(lambda x: other(self(x)))

f = Function(lambda x: x**2)
g = Function(lambda x: np.exp(-x))

h = f | g

plt.plot(x, h(x))

In [ ]:
class Function(object):
    def __init__(self, expression):
        self.expression = expression
    def __call__(self, x):
        return self.expression(x)
    def __or__(self, other):
        return Function(lambda x: other(self(x)))
    def derivative(self, x, dx=0.01):
        dy = self(x + 0.5 * dx) - self(x - 0.5 * dx)
        return dy / dx

f = Function(lambda x: x**2)
g = Function(lambda x: np.exp(-x))

h = f | g

plt.plot(x, h(x), label='$f \circ g$')
plt.plot(x, h.derivative(x), label='$(f \circ g)^\prime$')
plt.legend(fontsize=15)

## Iterators and generators
We revisit the Fibonacci problem and reformulate the solution using an `iterator`:

In [ ]:
class Fibonacci(object):
    def __init__(self, stop):
        self.stop = stop
        self.a, self.b = 1, 1
    def __iter__(self):
        return self
    def __next__(self):
        self.a, self.b = self.a + self.b, self.a
        if self.a < self.stop:
            return self.a
        raise StopIteration

fib = Fibonacci(100)
print(fib)

In [ ]:
for value in fib:
    print(value)

In [ ]:
print(list(Fibonacci(100)))

In [ ]:
fib = Fibonacci(100)
for i in range(10):
    print(i, next(fib))

Hence, an `iterator` is a `class` which has the methods `__iter__()` and `__next__()`.

A `generator` behaves very similar but does not require to write a `class`. The key component here is the `yield` command:

In [ ]:
def fibonacci(stop):
    a, b = 1, 1
    while True:
        a, b = a + b, a
        if a < stop:
            yield a
        else:
            break

print(fibonacci(100))

In [ ]:
for value in fibonacci(100):
    print(value)

In [ ]:
print(list(fibonacci(100)))

In [ ]:
fib = fibonacci(100)
for i in range(10):
    print(i, next(fib))